<div dir="rtl" align="right">

# 📘 L0_1 — S2 (Lesson Only): ارزیابی پایه — `accuracy_score`، `confusion_matrix`، `classification_report`

**Roadmap (L0):**  
L0_0 ✅ → **L0_1 (الان)** → L0_2 → L0_3 → L0_4

**جریان افقی (Sها):**  
Load → Frame → X,y → S1 ✅ → **S2 (الان)** → S3 → S4

</div>

<div dir="rtl" align="right">

## 🔗 پیوست: خلاصهٔ درس S1 (بدون تمرین) — انتخاب امن با `loc`/`iloc` + جداسازی `X,y`  (#S1-ATTACH)

### تفاوت بنیادی
| ویژگی | `loc` | `iloc` |
|---|---|---|
| مبنا | **برچسب (Label)** | **موقعیت عددی (Index Position)** |
| رفتار اسلایس | **شامل انتها** | **غیرشامل انتها** |
| ورودی | نام ستون/ایندکس، شرط، لیست | اعداد صحیح/بازه‌ها |
| شرط‌پذیری | بله (ماسک بولی) | خیر (پیش‌تبدیل لازم) |

**الگو:**  
```
df.loc[row_selector, col_selector]
df.iloc[row_index, col_index]
```

</div>

In [ ]:

# S1 Recap: Load iris and DataFrame
import pandas as pd
from sklearn.datasets import load_iris

iris = load_iris(as_frame=True)
df = iris.frame.copy()

feature_names = iris.feature_names
target_name = "target"


<div dir="rtl" align="right">

### نمونه‌های کلیدی S1
- `loc` برچسب‌محور (انتهای بازه شامل) و انتخاب شرطی
- `iloc` عدد-محور (انتهای بازه غیرشامل)
- تله: `df.loc['col']` انتخاب **سطر** است نه ستون

</div>

In [ ]:

# loc: سطرهای 0..4 و ستون‌های مشخص
ex_loc_1 = df.loc[0:4, ["sepal length (cm)", "target"]]

# loc شرطی: طول کاسبرگ > 5.5 برای دو ستون انتخابی
ex_loc_2 = df.loc[df["sepal length (cm)"] > 5.5, ["sepal length (cm)", "target"]]

# iloc: سطرهای 0..3 و ستون‌های 0 و 4
ex_iloc = df.iloc[0:4, [0, 4]]

ex_loc_1.head(), ex_loc_2.head(), ex_iloc


In [ ]:

# Trap: loc تک‌ورودی
trap_error = None
try:
    _ = df.loc['target']
except KeyError as e:
    trap_error = str(e)

# جداسازی استاندارد X,y
X = df[feature_names]    # DataFrame
y = df[target_name]      # Series

trap_error, X.head(), y.head()


<div dir="rtl" align="right">

## 🎯 هدف S2
- ساخت مدل پایه KNN (بدون Scaling) برای ارزیابی اولیه
- فهم صحیح سه متریک: `accuracy_score`، `confusion_matrix`، `classification_report`
- آماده‌سازی ذهنی برای S3 (Pipeline تمیز با `StandardScaler → KNN`)

</div>

<div dir="rtl" align="right">

## #S2-00 آماده‌سازی: Split استاندارد
- استفاده از `stratify=y` برای حفظ نسبت کلاس‌ها در Train/Test
- `random_state=42` برای تکرارپذیری

</div>

In [ ]:

from sklearn.model_selection import train_test_split

X_tr, X_te, y_tr, y_te = train_test_split(
    X, y, test_size=0.25, random_state=42, stratify=y
)

X_tr.shape, X_te.shape, y_tr.shape, y_te.shape


<div dir="rtl" align="right">

## #S2-01 مدل پایه KNN (بدون Scaling)
- KNN به فاصله حساس است؛ در S3 نشان می‌دهیم چرا Scaling مهم است.

</div>

In [ ]:

from sklearn.neighbors import KNeighborsClassifier

knn5 = KNeighborsClassifier(n_neighbors=5)
knn5.fit(X_tr, y_tr)
y_pred = knn5.predict(X_te)

y_pred[:10]


<div dir="rtl" align="right">

## #S2-02 دقت (Accuracy)
- تعریف: نسبت پیش‌بینی‌های درست به کل
- `normalize=False` تعداد درست‌ها را می‌دهد (نه نسبت)

</div>

In [ ]:

from sklearn.metrics import accuracy_score

acc = accuracy_score(y_te, y_pred)
acc_count = accuracy_score(y_te, y_pred, normalize=False)
len_test = len(y_te)

print("Accuracy:", acc)
print(f"Correct predictions: {acc_count} / {len_test}")


<div dir="rtl" align="right">

## #S2-03 ماتریس سردرگمی (Confusion Matrix)
- ردیف‌ها: کلاس واقعی — ستون‌ها: کلاس پیش‌بینی‌شده
- `normalize`: `'true'|'pred'|'all'`

</div>

In [ ]:

from sklearn.metrics import confusion_matrix
import numpy as np
import pandas as pd

labels = list(range(len(iris.target_names)))  # [0,1,2]

cm_raw = confusion_matrix(y_te, y_pred, labels=labels)
cm_true = confusion_matrix(y_te, y_pred, labels=labels, normalize='true')
cm_pred = confusion_matrix(y_te, y_pred, labels=labels, normalize='pred')
cm_all  = confusion_matrix(y_te, y_pred, labels=labels, normalize='all')

# نمایش خواناتر با برچسب نام کلاس‌ها
df_cm_raw  = pd.DataFrame(cm_raw, index=iris.target_names, columns=iris.target_names)
df_cm_true = pd.DataFrame(np.round(cm_true, 3), index=iris.target_names, columns=iris.target_names)
df_cm_pred = pd.DataFrame(np.round(cm_pred, 3), index=iris.target_names, columns=iris.target_names)
df_cm_all  = pd.DataFrame(np.round(cm_all, 3),  index=iris.target_names, columns=iris.target_names)

df_cm_raw, df_cm_true, df_cm_pred, df_cm_all


<div dir="rtl" align="right">

### نمودار سادهٔ CM نرمال‌شده (به‌ازای کلاس واقعی)
</div>

In [ ]:

import matplotlib.pyplot as plt

plt.figure()
plt.imshow(cm_true)
plt.title("Confusion Matrix (normalize='true')")
plt.xlabel("Predicted")
plt.ylabel("True")
plt.xticks(ticks=[0,1,2], labels=iris.target_names, rotation=45, ha='right')
plt.yticks(ticks=[0,1,2], labels=iris.target_names)
plt.colorbar()
plt.tight_layout()


<div dir="rtl" align="right">

## #S2-04 گزارش طبقه‌بندی (Classification Report)
- `precision`، `recall`، `f1-score` و `support` برای هر کلاس
- `digits` قالب نمایش را کنترل می‌کند
- در صورت خطر تقسیم صفر، از `zero_division=0` استفاده کن

</div>

In [ ]:

from sklearn.metrics import classification_report

cr = classification_report(
    y_te, y_pred,
    target_names=iris.target_names,
    digits=3
)
print(cr)


<div dir="rtl" align="right">

## #S2-05 الگوی تمیز ارزیابی (برای استفادهٔ تکراری)
</div>

In [ ]:

from typing import Dict, List
from sklearn.base import ClassifierMixin

def evaluate_model(
    clf: ClassifierMixin,
    X_test: pd.DataFrame,
    y_test: pd.Series,
    label_names: List[str]
) -> Dict[str, object]:
    y_pred_local = clf.predict(X_test)
    acc_local = accuracy_score(y_test, y_pred_local)
    cm_local = confusion_matrix(y_test, y_pred_local, labels=list(range(len(label_names))))
    cr_local = classification_report(y_test, y_pred_local, target_names=label_names, digits=3)
    return {"accuracy": acc_local, "cm": cm_local, "report": cr_local}

res = evaluate_model(knn5, X_te, y_te, list(iris.target_names))
print("Accuracy:", res["accuracy"])
print("Confusion matrix:\n", res["cm"])
print("Classification report:\n", res["report"])


<div dir="rtl" align="right">

## ⏭ ادامه (S3)
- در S3، همین زنجیره را با **Pipeline تمیز** ادامه می‌دهیم: `StandardScaler → KNN`  
- هدف: جلوگیری از *Data Leakage* و نشان دادن حساسیت KNN به مقیاس ویژگی‌ها

</div>